In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.8 MB/s eta 0:00:00


In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [23]:
from datasets.load import load_from_disk
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.optim as optim
from tqdm.notebook import tqdm
import torch.nn.functional as F
import csv
from transformers import TextDataset
from datasets import load_dataset, Dataset, concatenate_datasets, ClassLabel, Features, Value

# # Set the device
# if torch.cuda.is_available():    
#     device = torch.device("cuda")
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
#     print('We will use the GPU:', torch.cuda.get_device_name(0))
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from datasets import load_dataset

# Load RTE dataset
RTE = load_dataset("glue", "rte")

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
RTE

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [7]:
RTE.save_to_disk('/content/drive/MyDrive/CS685/project/datasets/RTE')

Saving the dataset (0/1 shards):   0%|          | 0/2490 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/277 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

# Augmentation

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
knowledge_dataset_params = {"RTE": {"n_iter": 10, "p_mask":0.1, "p_pos": 0.1, "p_ng":0.25}}

In [10]:
import transformers

# Tokenize the input sentence
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

In [11]:
# Load word distributions from disk
word_distributions = torch.load('/content/drive/MyDrive/CS685/project/word_distributions.pt')

In [12]:
# Define a function to replace a single word with a random word of the same part-of-speech tag
def replace_word_with_random_word_of_same_pos(word, pos_tag):
    # Get the unigram word distribution for the given part-of-speech tag
    word_distribution = word_distributions[pos_tag]
    
    # Sample a new word from the distribution
    new_word_index = torch.multinomial(word_distribution, 1).item()
    new_word = tokenizer.convert_ids_to_tokens([new_word_index])[0]
    
    return new_word

In [13]:
def pos_guided_word_replacement(sentence, word_to_replace):
  # Tokenize the sentence
  tokenized_sentence = tokenizer.tokenize(sentence)

  pos_tag_to_replace = nltk.pos_tag([word_to_replace])[0][1]

  for i in range(len(tokenized_sentence)):
    if tokenized_sentence[i] == word_to_replace:
        return replace_word_with_random_word_of_same_pos(word_to_replace, pos_tag_to_replace)

In [14]:
cur_train = RTE['train']

In [17]:
from collections import defaultdict
pos_tag_word_map = defaultdict(lambda: set())
with torch.no_grad(), tqdm(total=len(cur_train['sentence1']), desc="Computing the word distributions with sentence1") as pbar:
  for s in cur_train['sentence1']:
    for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(s)):
      pos_tag_word_map[pos_tag].add(word)
    pbar.update(1)
with torch.no_grad(), tqdm(total=len(cur_train['sentence2']), desc="Computing the word distributions with sentence2") as pbar:
  for s in cur_train['sentence2']:
    for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(s)):
      pos_tag_word_map[pos_tag].add(word)
    pbar.update(1)

Computing the word distributions with sentence1:   0%|          | 0/2490 [00:00<?, ?it/s]

Computing the word distributions with sentence2:   0%|          | 0/2490 [00:00<?, ?it/s]

In [18]:
pos_tag_word_map_list = defaultdict(lambda: [])
for pos_tag in pos_tag_word_map:
  pos_tag_word_map_list[pos_tag] = list(pos_tag_word_map[pos_tag])

In [20]:
import random
data = {}
for column in cur_train.column_names:
  data[column] = []

idx_counter = len(cur_train)

error_count = 0

# Modifying the first sentence
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using first sentence") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['RTE']['n_iter']):
        res = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence1'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['RTE']['p_mask']:
            res.append('[MASK]')
          elif knowledge_dataset_params['RTE']['p_mask'] <= X < knowledge_dataset_params['RTE']['p_mask']+knowledge_dataset_params['RTE']['p_pos']:
            res.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['RTE']['p_ng']:
          synthetic_text1 = ' '.join(random.sample(res, min(len(res), random.randint(1, 5))))
          data['sentence1'].append(synthetic_text1)
          data['sentence2'].append(row['sentence2'])
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

# Modifying the second sentence
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using second sentence") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['RTE']['n_iter']):
        res = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence2'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['RTE']['p_mask']:
            res.append('[MASK]')
          elif knowledge_dataset_params['RTE']['p_mask'] <= X < knowledge_dataset_params['RTE']['p_mask']+knowledge_dataset_params['RTE']['p_pos']:
            res.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['RTE']['p_ng']:
          synthetic_text2 = ' '.join(random.sample(res, min(len(res), random.randint(1, 5))))
          data['sentence1'].append(row['sentence1'])
          data['sentence2'].append(synthetic_text2)
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

# Modifying both sentences
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using both sentences") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['RTE']['n_iter']):
        res1 = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence1'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['RTE']['p_mask']:
            res1.append('[MASK]')
          elif knowledge_dataset_params['RTE']['p_mask'] <= X < knowledge_dataset_params['RTE']['p_mask']+knowledge_dataset_params['RTE']['p_pos']:
            res1.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res1.append(word)
        res2 = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence2'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['RTE']['p_mask']:
            res2.append('[MASK]')
          elif knowledge_dataset_params['RTE']['p_mask'] <= X < knowledge_dataset_params['RTE']['p_mask']+knowledge_dataset_params['RTE']['p_pos']:
            res2.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res2.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['RTE']['p_ng']:
          synthetic_text1 = ' '.join(random.sample(res1, min(len(res1), random.randint(1, 5))))
          synthetic_text2 = ' '.join(random.sample(res2, min(len(res2), random.randint(1, 5))))
          data['sentence1'].append(synthetic_text1)
          data['sentence2'].append(synthetic_text2)
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

print('\nMissed Entries', str(error_count))

Computing the synthetic examples using first sentence:   0%|          | 0/2490 [00:00<?, ?it/s]

Computing the synthetic examples using second sentence:   0%|          | 0/2490 [00:00<?, ?it/s]

Computing the synthetic examples using both sentences:   0%|          | 0/2490 [00:00<?, ?it/s]


Missed Entries 0


In [21]:
from datasets import Dataset
new_train = Dataset.from_dict(data)

In [28]:
# schema of MRPC dataset
ds_schema = Features({
    "sentence1": Value("string"),
    "sentence2": Value("string"),
    "label": ClassLabel(names=['entailment', 'not_entailment'], id=None),
    "idx": Value(dtype='int32', id=None)
})

new_train = new_train.cast(ds_schema)

Casting the dataset:   0%|          | 0/18553 [00:00<?, ? examples/s]

In [29]:
cur_train[:10]

{'sentence1': ['No Weapons of Mass Destruction Found in Iraq Yet.',
  'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
  'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
  'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.',
  "A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keyworth, Nottinghamshire, on 30 October 1983 when she disappeared. Her body

In [30]:
knowledge_dataset = concatenate_datasets([cur_train, new_train])

In [31]:
RTE['train'] = knowledge_dataset

In [32]:
RTE.save_to_disk('/content/drive/MyDrive/CS685/project/datasets/RTE')

Saving the dataset (0/1 shards):   0%|          | 0/21043 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/277 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [33]:
RTE

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 21043
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [34]:
RTE['train'][:10]

{'sentence1': ['No Weapons of Mass Destruction Found in Iraq Yet.',
  'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
  'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
  'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.',
  "A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keyworth, Nottinghamshire, on 30 October 1983 when she disappeared. Her body